In [1]:
from time import sleep

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import pandas as pd


In [3]:
url_proger = 'https://tproger.ru/quiz/real-programmer'
result_file = './data/result_tproger.csv'

#Пустой датафрейм для хранения результата пагенации
df = pd.DataFrame(columns=('answer', '%', 'correctness'))
df.to_csv(result_file, index=False)

driver = webdriver.Chrome()
driver.maximize_window()

driver.get(url_proger)

#Ждем прогрузи страницы и скролим вниз чтобы появились кнопки опроса
sleep(3)

driver.execute_script("window.scrollTo(0, window.scrollY + window.innerHeight);")

# Заходим в цикл и скраппии данные со всех страниц теста
while True:
    try:
        # Кликаем первую кнопку для просмотра результатов
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'tp-quiz-answer-list__item'))
        )
        element.click()
        
        # Ждем обновления скриптов
        sleep(1)
        
        # Получаем правильный ответ и записываем его в строку датафрейма для данной страницы
        r_answer = driver.find_element(By.CLASS_NAME, 'tp-answer--result-true').text.split('\n')
        r_answer.append('correct')
        r_answer_final = pd.DataFrame([r_answer], columns=['answer', '%', 'correctness'])
        
        # Некоторые страницы хранят только 1 неправильный ответ, посему проходим по логике длинны элементов в сыром ответе
        if len(driver.find_elements(By.CLASS_NAME, 'tp-answer--result-false')) > 1:
            f_answers = driver.find_elements(By.CLASS_NAME, 'tp-answer--result-false')
            f_answers = [i.text.split('\n') + ['wrong answer'] for i in f_answers]
            f_answers_final = pd.DataFrame(f_answers, columns=['answer', '%', 'correctness'])
        else:
            f_answer = driver.find_elements(By.CLASS_NAME, 'tp-answer--result-false')[0].text.split('\n')
            f_answer.append('wrong answer')
            f_answers_final = pd.DataFrame([f_answer], columns=['answer', '%', 'correctness'])
        
        # Формируем итог по страницы и клеим к результирующему датафрейму
        result = pd.concat([r_answer_final, f_answers_final])
        final_df = pd.read_csv(result_file)
        final_df = pd.concat([final_df, result])
        final_df.reset_index(drop=True, inplace=True)
        final_df.to_csv(result_file, index=False)
        
        # Ищем кнопку далее, приходится брать по xpath ибо смежный класс кнопки имеет кнопка создать статью
        button_next = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, '//*[@id="quiz-app"]/div/div/div/div[3]/div[2]/button'))
        )
        
        # Пришлось реализовать клик через js скрипт ибо обычный клик не работал
        driver.execute_script("arguments[0].click();", button_next)
        sleep(3)
    
    # Из-за характера button_next приходится просто перехватывать TimeoutException
    except:
        break
    
display(final_df)


,answer,%,correctness
0,Hello world,91%,correct
1,Сортировку «пузырьком»,3%,wrong answer
2,Для взлома аккаунта «ВКонтакте». Такая програм...,2%,wrong answer
3,Это вопрос с подвохом. Начинают они с починки ...,4%,wrong answer
4,"Нет, это фантастика",71%,correct
5,"Да, конечно",29%,wrong answer
6,"Буду искать ошибку в компиляторе, где-то она д...",73%,correct
7,Выключу комп и спокойно пойду спать — дело сде...,8%,wrong answer
8,Порадуюсь за себя и продолжу писать код,19%,wrong answer
9,"Изменятся, конечно",80%,correct
